In [36]:
import sklearn
from IPython.core.display import HTML

HTML("<style>" + open("style.css").read() + "</style>")

<div class="headline">
Language Technology / Sprachtechnologie
<br><br>
Wintersemester 2019/2020
</div>
<br>
<div class="description">
    Übung zum Thema <i id="topic">"Named Entity / Coreference"</i>
    <br><br>
    Deadline Abgabe: <i #id="submission">Thursday, 14.11.2019 (23:55 Uhr)</i>
</div>

# Präsenzübung

In [37]:
import nltk
from nltk.probability import FreqDist
from nltk.probability import ConditionalFreqDist
from nltk.corpus import*
from nltk.book import*
from nltk.chunk import *
from nltk.chunk.util import *
from nltk.chunk.regexp import *

from sklearn import datasets, svm, tree, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd 
from nltk.tag import pos_tag
from nltk.corpus import gazetteers, names

### Warm Up

<div class="task_description">
    <i class="task">Task 4.1:</i> Named entitiy recognition: <br>
</div>

Which of the following statements are true?

1. The goal of a named entity recognition (NER) system is to identify all textual mentions of the named entities.
2. Named entity recognition is a method to extract person names from text.
3. Named entities are language independent.
4. In named entitiy recognition we need to be able to identify the beginning and the end of multi-token sequences.

<strong style="color: blue">Lösung:</strong>

1. True, a named entity recognition (NER) system should identify all textual mentions of the named entities.
2. True. PERSON is a valid named entity type, however the NE recognition systems may extract numerous other types as LOCATION or ORGANIZATION.
3. False. Although names of persons are usually not translated (besides the alphabet), currencies and dates can be specific to one language. Countries (GPE: geo-political entities) are sometimes translated (e.g. Elfenbeinküste - Ivory coast - Cote d'Ivoire) 
4. True, since named entities may consist of more than one word (e.g. George W. Bush)

</div>

### Using a Named Entities Classifier

A type of noun phrase that is of particular interest is a named entity. This might be a person, such as Albert Einstein, or a place, such as Duisburg or a business, such as Irish Pub. <br>
In general, this is a hard problem. Words can have multiple uses, and there’s an unbounded number of possible names. Within a domain, though, we can have better luck. NLTK provides a classifier that has already been trained to recognize named entities, accessed with the function nltk.ne_chunk() <br>
The table below states the commonly used types of named entities, as they are provided by nltk:

| NE Type | Examples  |
|------|------|
|ORGANIZATION|Georgia-Pacific Corp., WHO
|PERSON| Eddy Bonte, President Obama
|LOCATION|Murray River, Mount Everest
|DATE|June, 2008-06-29
|TIME|two fifty a m, 1:30 p.m.
|MONEY|175 million Canadian Dollars, GBP 10.40
|PERCENT|twenty pct, 18.75%
|FACILITY|Washington Monument, Stonehenge
|GPE (geo-political entities)|South East Asia, Midlothian


<div class="task_description">
    <i class="task">Task 4.2:</i> <br>
</div>

Use the sentence below: <br><br>
The capital of the United States of America is named after the first US president George Washington.

<div class="task_description">
   <i class="subtask">4.2.1</i> <i class="l2">L2</i> <br>
</div>

Use word_tokenize to tokenize the sample.

<strong style="color: blue">Lösung:</strong>

We tokenize the above text using the expression nltk.word_tokenize(sentence). The tokenizer outputs the list of tokens.

In [4]:
nltk.word_tokenize("The capital of the United States of America is named after the first US president George Washington.")

['The',
 'capital',
 'of',
 'the',
 'United',
 'States',
 'of',
 'America',
 'is',
 'named',
 'after',
 'the',
 'first',
 'US',
 'president',
 'George',
 'Washington',
 '.']

<div class="task_description">
   <i class="subtask">4.2.2</i> <i class="l2">L2</i> <br>
</div>

Use nltk.pos_tag to tag the sentence.

<strong style="color: blue">Lösung:</strong>

We assign part-of-speech tags to the above text using the following code.

In [5]:
sentence = nltk.word_tokenize("The capital of the United States of America is named after the first US president George Washington.")
sentence = nltk.pos_tag(sentence)
print (sentence)

[('The', 'DT'), ('capital', 'NN'), ('of', 'IN'), ('the', 'DT'), ('United', 'NNP'), ('States', 'NNPS'), ('of', 'IN'), ('America', 'NNP'), ('is', 'VBZ'), ('named', 'VBN'), ('after', 'IN'), ('the', 'DT'), ('first', 'JJ'), ('US', 'NNP'), ('president', 'NN'), ('George', 'NNP'), ('Washington', 'NNP'), ('.', '.')]


<div class="task_description">
   <i class="subtask">4.2.3</i> <i class="l2">L2</i> <br>
</div>

Use nltk.ne_chunk to chunk the tagged sentence. Experiment with the argument "binary". What is the difference?

<strong style="color: blue">Lösung:</strong>

We extract named entities from the above text using the following code:

In [20]:
tree = nltk.ne_chunk(sentence, binary=False)
print (tree)

(S
  The/DT
  capital/NN
  of/IN
  the/DT
  (GPE United/NNP States/NNPS)
  of/IN
  (GPE America/NNP)
  is/VBZ
  named/VBN
  after/IN
  the/DT
  first/JJ
  (GPE US/NNP)
  president/NN
  (PERSON George/NNP Washington/NNP)
  ./.)


We notice that setting the 'binary' parameter to True outputs named entities without types. For example the excerpt "('America','NNP')" labels America as a Named Entity. Whereas, the default setting of the 'binary' parameter to False enables the system to output Named Entity types.

<div class="task_description">
   <i class="subtask">4.2.4</i> <i class="l2">L2</i> <br>
</div>

Draw (.draw()) and analyze the resulting tree structure

<strong style="color: blue">Lösung:</strong>

In [22]:
import tkinter
tree = nltk.ne_chunk(sentence, binary=True)
tree.draw()

ModuleNotFoundError: No module named '_tkinter'

We notice that the named entities are extracted: United States, America, US and George Washington. <br>
Also the tree structure is very flat, with the bottom layer of extracted entities.

<div class="task_description">
   <i class="subtask">4.2.5</i> <i class="l3">L3</i> <br>
</div>

Write a function extract_entity_names(tree), that extracts all identified named entities of the given tree and returns it as a list of words.<br>
Since 'tree' is is a nested structure implement this function using a recursion. It is standard to use recursice function to traverse a tree. The listing below defines an algorithm to traverse a tree. You may change it to fit your purpose.

In [16]:
def traverse(t):
    try:
        t.label
    except AttributeError:
        print (t)
    else:
        #Now we know that t.node is defined
        print ('(', t.label),
        for child in t:
            traverse(child)
        print (')'),
        
traverse(tree)



( <bound method Tree.label of Tree('S', [('The', 'DT'), ('capital', 'NN'), ('of', 'IN'), ('the', 'DT'), Tree('GPE', [('United', 'NNP'), ('States', 'NNPS')]), ('of', 'IN'), Tree('GPE', [('America', 'NNP')]), ('is', 'VBZ'), ('named', 'VBN'), ('after', 'IN'), ('the', 'DT'), ('first', 'JJ'), Tree('GPE', [('US', 'NNP')]), ('president', 'NN'), Tree('PERSON', [('George', 'NNP'), ('Washington', 'NNP')]), ('.', '.')])>
('The', 'DT')
('capital', 'NN')
('of', 'IN')
('the', 'DT')
( <bound method Tree.label of Tree('GPE', [('United', 'NNP'), ('States', 'NNPS')])>
('United', 'NNP')
('States', 'NNPS')
)
('of', 'IN')
( <bound method Tree.label of Tree('GPE', [('America', 'NNP')])>
('America', 'NNP')
)
('is', 'VBZ')
('named', 'VBN')
('after', 'IN')
('the', 'DT')
('first', 'JJ')
( <bound method Tree.label of Tree('GPE', [('US', 'NNP')])>
('US', 'NNP')
)
('president', 'NN')
( <bound method Tree.label of Tree('PERSON', [('George', 'NNP'), ('Washington', 'NNP')])>
('George', 'NNP')
('Washington', 'NNP')
)


<strong style="color: blue">Lösung:</strong>

Refer to the code in the following listing:

In [23]:
def extract_entity_names(tree):
    '''This function exctracts all named entities from a tree. 
    If a tree node has the label 'NE' it extracts all values of the leaves and adds it to the set of named entities
    This named entity is coverted to a list and returned'''
    
    entity_names = []
    try:
        if tree.label() == 'NE':
            entity = ' '.join(leaf[0] for leaf in tree.leaves())
            entity_names.append(entity)
    except AttributeError:
        pass
    else:
        for child in tree:
            entity = extract_entity_names(child)
            if len(entity)>0:
                entity_names.append(entity)
    return entity_names
    

text = "The capital of the United States is named after the first US president George Washington"

sentence = nltk.word_tokenize(text)
sentence = nltk.pos_tag(sentence)
tree = nltk.ne_chunk(sentence, binary = True)

print ("Tree: \n",tree)
print ("Named entities: ",extract_entity_names(tree))

Tree: 
 (S
  The/DT
  capital/NN
  of/IN
  the/DT
  (NE United/NNP States/NNPS)
  is/VBZ
  named/VBN
  after/IN
  the/DT
  first/JJ
  (NE US/NNP)
  president/NN
  (NE George/NNP Washington/NNP))
Named entities:  [['United States'], ['US'], ['George Washington']]


### Precision, Recall, F-Score

<div class="task_description">
    <i class="task">Task 4.3:</i> <br>
</div>

The following confusion matrix shows the evaluation result of a named entities classifier. The columns contain the gold standard and the rows the system output. The target class is NE.

Confusion Matrix |NE | no NE |
-|-|-|
NE| 50 | 30 |
no NE| 20 | 200 |

<div class="task_description">
   <i class="subtask">4.3.1</i> <i class="l1">L1</i> <br>
</div>
How many true positives, true negatives, false positives and false negatives are there? How do you interpret them?

<strong style="color: blue">Lösung:</strong>

There are 50 true positives (System and Gold say "NE"), and 200 true negatives (System and Gold say "no NE"): These are the cases where the system made the correct decision.

There are 30 false positives (System says "NE" but correct would be "no NE"), and 20 false negatives (System says "no NE" but correct would be "NE"): These are the cases where the system made an incorrect decision.


<div class="task_description">
   <i class="subtask">4.3.2</i> <i class="l2">L2</i> <br>
</div>
Compute precision, recall and F-score given the confusion matrix above.

<strong style="color: blue">Lösung:</strong>

In [ ]:
precision = 50/(50+30)
recall = 50/(50+20)

print("Precision:", precision)
print("Recall:", recall)
print("F-Score: ", 2*precision*recall/(precision+recall) )

### Building your own Named Entities Classifier

<div class="task_description">
    <i class="task">Task 4.4:</i> <br>
</div>

<div class="task_description">
   <i class="subtask">4.4.1</i> <i class="l1">L1</i> <br>
</div>
What does the following code do?

In [24]:
df = pd.read_csv("NER_clean.csv", delimiter = "\t", encoding="utf-8", names=["WORD", "NE"], quoting=3)
df["WORD"] = df["WORD"].apply(str)
print(df[:30])

         WORD     NE
0           &  False
1          gt  False
2           ;  False
3           *  False
4         The  False
5     soldier  False
6         was  False
7      killed  False
8        when  False
9     another  False
10  avalanche  False
11        hit  False
12         an  False
13       army  False
14   barracks  False
15         in  False
16        the  False
17   northern  False
18       area  False
19         of  False
20    Sonmarg   True
21          ,  False
22       said  False
23          a  False
24   military  False
25  spokesman  False
26          .  False
27          &  False
28         gt  False
29          ;  False


<strong style="color: blue">Lösung:</strong>

A csv file is read in and converted to a data frame with two columns: One contains the word and the other one a binary distinction whether the word is a named entity ("True") or not ("False"). (quoting = 3 is necessary to also read in quotation marks as regular tokens).
The second line of code only makes sure that anything in the first column is treated as a string (even numbers). Finally, the first 30 rows of the data frame are printed.

<div class="task_description">
   <i class="subtask">4.4.2</i> <i class="l1">L1</i> <br>
</div>
What does the following code do?

In [25]:
words =list(df.loc[:, "WORD"])
df["WORDLENGTH"] = [len(word) for word in words]
print(df[:30])

         WORD     NE  WORDLENGTH
0           &  False           1
1          gt  False           2
2           ;  False           1
3           *  False           1
4         The  False           3
5     soldier  False           7
6         was  False           3
7      killed  False           6
8        when  False           4
9     another  False           7
10  avalanche  False           9
11        hit  False           3
12         an  False           2
13       army  False           4
14   barracks  False           8
15         in  False           2
16        the  False           3
17   northern  False           8
18       area  False           4
19         of  False           2
20    Sonmarg   True           7
21          ,  False           1
22       said  False           4
23          a  False           1
24   military  False           8
25  spokesman  False           9
26          .  False           1
27          &  False           1
28         gt  False           2
29        

<strong style="color: blue">Lösung:</strong>
    
A third column is added which contains the length the token.

<div class="task_description">
   <i class="subtask">4.4.3</i> <i class="l2">L2</i> <br>
</div>

Add 4 columns to the data frame which contain

- whether a word is capitalized (True/False)
- whether a word is fully written in uppercase (True/False)
- whether the word is a noun (True/False)
- whether the word appears in the corpus "names" or "gazetteers" from NLTK (True/False)

<strong style="color: blue">Lösung:</strong>

In [26]:
df["IS_CAPITALIZED"] = [word.istitle() for word in words]
df["IS_UPPER"] = [word.isupper() for word in words]

In [33]:
print(df[:30])

         WORD     NE  WORDLENGTH  IS_CAPITALIZED  IS_UPPER  IN_LIST   NOUN
0           &  False           1           False     False    False  False
1          gt  False           2           False     False    False   True
2           ;  False           1           False     False    False  False
3           *  False           1           False     False    False  False
4         The  False           3            True     False    False  False
5     soldier  False           7           False     False    False   True
6         was  False           3           False     False    False  False
7      killed  False           6           False     False    False  False
8        when  False           4           False     False    False  False
9     another  False           7           False     False    False  False
10  avalanche  False           9           False     False    False   True
11        hit  False           3           False     False    False  False
12         an  False     

In [30]:
set_gaz = set(gazetteers.words())
set_names = set(names.words())
df["IN_LIST"] = [word in set_gaz or word in set_names for word in words]

In [32]:
pos = pos_tag(words, tagset="universal")
df["NOUN"] = [tag == "NOUN" for (word, tag) in pos]

In [ ]:
print(df[:10])

<div class="task_description">
   <i class="subtask">4.4.4</i> <i class="l3">L3</i> <br>
</div>

The following code creates a Decision Tree Classifier which classifies whether a token is a named entity or not based on the features you provided above. The data are split in a training and a test set and the variable "predicted" contains the predicted labels (NE = "True", no NE ="False") while "gold" contains the corresponding gold labels.

In [40]:
x = df.iloc[:, 2:len(df.columns)]
y = df.iloc[:, [1]]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

import sklearn

c_tree = sklearn.tree.DecisionTreeClassifier(max_depth=4)
c_tree.fit(x_train, y_train)

predicted = list(c_tree.predict(x_test))
gold = list(y_test.loc[:, "NE"])

In [50]:
print(df)
print(x)
print(y)

        WORD     NE  WORDLENGTH  IS_CAPITALIZED  IS_UPPER  IN_LIST   NOUN
0          &  False           1           False     False    False  False
1         gt  False           2           False     False    False   True
2          ;  False           1           False     False    False  False
3          *  False           1           False     False    False  False
4        The  False           3            True     False    False  False
...      ...    ...         ...             ...       ...      ...    ...
23389   with  False           4           False     False    False  False
23390   this  False           4           False     False    False  False
23391  dress  False           5           False     False    False   True
23392   code  False           4           False     False    False   True
23393      😂  False           1           False     False    False   True

[23394 rows x 7 columns]
       WORDLENGTH  IS_CAPITALIZED  IS_UPPER  IN_LIST   NOUN
0               1         

Based on the predicted and the gold labels, compute precision, recall and F-score for the classifier. 

<strong style="color: blue">Lösung:</strong>

In [51]:
tp = sum(pair == (True, True) for pair in zip(predicted, gold))
fp = sum(pair == (True, False) for pair in zip(predicted, gold))
fn = sum(pair == (False, True) for pair in zip(predicted, gold))

precision = tp/(tp+fp)
recall = tp/(tp+fn)
fscore = 2*precision*recall/(precision+recall)

print("precision:", precision)
print("recall:", recall)
print("f-score:", fscore)

precision: 0.7884615384615384
recall: 0.10875331564986737
f-score: 0.19114219114219116


You can compare your results to the built-in classification report:

In [52]:
print(classification_report(gold,predicted))

              precision    recall  f1-score   support

       False       0.93      1.00      0.96      4302
        True       0.79      0.11      0.19       377

    accuracy                           0.93      4679
   macro avg       0.86      0.55      0.58      4679
weighted avg       0.92      0.93      0.90      4679



# Homework

<div class="task_description">
    <i class="task">Task 4.1:</i> 
</div>

<i class="subtask">4.1.1</i> 
Annotate all named entities in the file "Langtech_NER.txt"

* The file contains 100 German sentences (note that the sentences do not form a coherent text) and each sentence may contain one or more named entities but it is also possible that there is no named entity in a sentence
<br><br>
* The 4 named entity types to annotate are PERSON (PER), ORGANIZATION (ORG), LOCATION (LOC), OTHER (OTH). For further information about which named entity belongs to which type, please refer to the "NoSta-D-TagSet" on page 6 in the file "Clarin_NoSta-D_NER-AnnotationGuidelines.pdf" that you can download from Moodle. Important: you are not asked to follow these annotation guidelines completely. Especially, note the followning:
     * Anything that is tagged with "deriv" or "part" tags according to these guidelines is ignored (e.g. LOCderiv, ORGpart)
     * In our annotation, there are no nested named entities. For example "Bayern München" is labeled as ORG and the individual parts "Bayern" and "München" are not labeled as LOC. As a general rule, the longest possible span gets the label.
 <br>	
<br>
* Upload the annotated file (ending with ".ann", see below), to Moodle. Make sure that the filename contains your name!

<i class="subtask">4.1.2</i> 

Write down at least 5 different cases that you found difficult to annotate. For each, write down 1-2 sentences explaining why it was difficult (e.g. by saying which other label could have applied and why or why you were unsure whether something is a named entity or not). Upload your descriptions to Moodle as a PDF file.



### Technical instructions


- Download the annotation tool YEDDA from https://github.com/jiesutd/YEDDA

- Attention: YEDDA requires Python 2.7, so make sure you have this version installed!

- To start the annotation, open a console (in the YEDDA-master folder) and type python YEDDA.py (make sure you start it with Python 2 not Python 3, so maybe you have to type something like /path/to/python2 YEDDA.py !)

- Download the file "Langtech_NER.config" from Moodle and place it in the folder "YEDDA-master/configs/".

- To open the sentences to annotate, click on "open" and select the file "Langtech_NER.txt" (or "Langtech_NER.ann" if you have already saved an annotated version and want to continue)

- Select the correct set of labels: In the drop down menu under "Map Templates" on the right hand side, select the file "Langtech_NER.config"

- To annotate a named entity, mark the whole Named Entity and press the key on the keyboard that is associated with the right label (A: PERSON, B: ORGANISATION, C:LOCATION, D:OTHER)

- To change a label, click within the entity span and press the key for the new label

- To remove a label, click within the entity span and press 'q' . Important: In order to remove the label, do not mark the whole entity. If you then press 'q' this will remove the whole entity, not just the label!

- Clicking on "Export" will save the annotated text. Note: two files will be saved, one ending with *.ann and one with *.anns. Upload the one ending with *.ann to Moodle (change the filename so that it contains your name!)